In [2]:
%matplotlib inline
import pandas as pd
import json
#from pprint import pprint
#from random import sample
#from nltk.tokenize import word_tokenize
#import nltk
#import seaborn as sns
import numpy as np

In [3]:
def json_df(datapass):
    '''
    Load the json file and parse the file to pandas dataframe format
    
    Input:
        datapass(str) : directory to the json file
    Output:
        df(dataframe) : pandas dataframe object
    '''
    
    data = [] 
    with open(datapass, 'r', encoding='utf-8') as data_file: 
        for f in data_file:
            data.append(json.loads(f))
    df = pd.DataFrame(data)
    return df

#business and review json files --> dataframes

DATAPASS1 = 'C:\\Users\\cdchang\\Downloads\\yelp_dataset\\dataset\\business.json'
business = json_df(DATAPASS1)
DATAPASS2 = 'C:\\Users\\cdchang\\Downloads\\yelp_dataset\\dataset\\review.json'
reviews = json_df(DATAPASS2)

In [5]:
'''
TODO: come up with function that outputs the unique business ids from different
      categories found in the business json

first thing just look at business
which subcategories have more reviews
(cross reference)

group by subcategories
list of business IDs
cross reference, with review files
which categories have more data

CATEGORY --> BUSINESS IDS

'''

#business.head(3)

#category_all

#1. create a dictionary with each entry's set of categories
# specifically, create the appropriate keys and values 
#2. 

In [12]:
category_list = business[business.columns[3]]
category_all = category_list
#category head count
def category_head_count(cat):
    '''
    generate head counts of all different categories, then
    return total number of occurences of that specific category
    
    Input:
        cat(str) : category phrase
    Output:
        by_category[cat] : frequency of category
    '''
    
    by_category = {}
    for word_string in category_all:
        for indiv in word_string:
            if indiv not in by_category:
                by_category[indiv] = 1
            else:
                by_category[indiv]+=1
    
    return by_category[cat]

#example
#category_head_count('Shopping')

def category_count(df):
    '''
    return a dictionary that maps category to the unique business count
    Input:
        df(dataframe) : business dataframe with the column name 'categories'
    Output:
        categories(list) : sorted list of tuple ('category', business)
    '''
    categories = {}
    for C in df.categories:
        for c in C:
            if c in categories:
                categories[c] += 1
            else:
                categories[c] = 1
                
    categories = sorted(categories.items(), key=lambda x: x[1], reverse = True)
    return categories

category_count(business)

[('Restaurants', 51613),
 ('Shopping', 24595),
 ('Food', 23014),
 ('Beauty & Spas', 15139),
 ('Home Services', 13202),
 ('Health & Medical', 12033),
 ('Nightlife', 11364),
 ('Bars', 9868),
 ('Automotive', 9476),
 ('Local Services', 9343),
 ('Event Planning & Services', 8038),
 ('Active Life', 7427),
 ('Fashion', 6299),
 ('Sandwiches', 5864),
 ('Fast Food', 5792),
 ('American (Traditional)', 5737),
 ('Pizza', 5652),
 ('Coffee & Tea', 5565),
 ('Hair Salons', 5395),
 ('Hotels & Travel', 5188),
 ('Arts & Entertainment', 5054),
 ('Home & Garden', 4584),
 ('Auto Repair', 4480),
 ('Italian', 4411),
 ('Burgers', 4236),
 ('Doctors', 4124),
 ('Breakfast & Brunch', 4103),
 ('Mexican', 3913),
 ('Nail Salons', 3884),
 ('Professional Services', 3865),
 ('American (New)', 3802),
 ('Chinese', 3775),
 ('Real Estate', 3729),
 ('Specialty Food', 3620),
 ('Fitness & Instruction', 3615),
 ('Pets', 3153),
 ('Grocery', 3044),
 ('Bakeries', 3014),
 ('Cafes', 2812),
 ('Hair Removal', 2704),
 ('Dentists', 2683)

In [ ]:
# approach: find overlapping word distributions per category

def consolidating_categories(df):
    categories = category_count(business)
    print(categories.keys())

In [5]:
from collections import defaultdict
def business_id_retrieval(cat):
    '''
    
    Input:
        cat(str) : category 
    Output:
       id_list[cat] : business ids (values) of a particular category (key)
    '''
    id_list = dict()
    idx = 0
    for word_string in category_all:
        for indiv in word_string:
            if indiv not in id_list:
                id_list[indiv] = []
            else:
                id_list[indiv].append(business.business_id[idx])
        idx+=1

    return id_list[cat]

#business_id_retrieval("Shopping")

def business_id_retrieval(cat, business):

    '''
    Input:
        cat(str) : category
        business(dataframe) : the business data
    Output:
        id_list(set) : business ids of a particular category
    '''

    id_list = set()
    idx = 0
    for row in business.values: #for every row
        categories = row[3]     #third entry in each row
        if cat in categories:
            id_list.add(row[2])
    return id_list

#business_id_retrieval("Shopping")
#business_id_retrieval("Shopping", business)

In [ ]:
#link to review json 
from collections import defaultdict

#miscellaneous methods for retrieving 

def get_reviews_idonly(b_id): #get review id list from a single business ID
    '''
    input: business_id
    output: array of reviews associated with that particular business_id
    '''
    review_list = set()
    for row in reviews.values:
        b = row[0]
        if (b_id == b):
            
            review_list.add(row[4])
    return review_list
    
def get_reviews(cat, business): #get review list indices from review json for a single category
    '''
    input: category phrase, business dataframe
    output: array of indices in review json associated to a particular category
    '''
    
    array = []
    all_reviews = set()
    cat_id_list = business_id_retrieval(cat, business)
    existing_ids = reviews['business_id']

    
    segments = dict(list(reviews.groupby('business_id')))
    #for i in cat_id_list:
    #    if i in existing_ids:
    #        array.add(segments[i])
    
    return segments

segments = dict(list(reviews.groupby('business_id')))
print(segments)
#unique classification issue - i.e. "Shopping" & "Shopping Centers" fall under
#different categories 

In [8]:
#DATA RICH CLASSIFICATIONS
reviews.values[0]
reviews.values[1]

reviews.head(3)[0]

KeyError: 0

In [3]:
def year_distrib(b_id, reviews):
    
    #get_reviews(b_id)
    
    segments = dict(list(reviews.groupby('business_id')))
    reviews = segments[b_id]
    dates = {}
    for row in reviews:
        date = row[2][0:4]
        
        if date not in dates:
            dates[date] = 1
        else:
            dates[date] +=1
    
    return dates

year_distrib("uYHaNptLzDLoV_JZ_MuzUA", reviews)

In [4]:
#def year_distrib(cat, reviews):

hello world
